Named Entity Recognition 

Johnathan Zhao, Wensheng Lin, Fasola Ibukunoluwa, Hyunje Sung 

In [1]:
# Imported libraries 
import spacy
# import matplotlib
# import numpy as np
import pandas as pd 
import requests
import os
from tqdm import tqdm
from spacy import displacy
# from spacy.scorer import Scorer
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split
!python -m spacy download en_core_web_lg  

2022-12-05 04:28:40.249048: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 17 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


# **Loading Data**



In [2]:
news=pd.read_csv("abcnews-date-text.csv")
news.dropna()
news

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1244179,20211231,two aged care residents die as state records 2...
1244180,20211231,victoria records 5;919 new cases and seven deaths
1244181,20211231,wa delays adopting new close contact definition
1244182,20211231,western ringtail possums found badly dehydrate...


In [3]:
headlines=news.headline_text[0:500]       #Store headlines into this list 
headlines

0      aba decides against community broadcasting lic...
1         act fire witnesses must be aware of defamation
2         a g calls for infrastructure protection summit
3               air nz staff in aust strike for pay rise
4          air nz strike to affect australian travellers
                             ...                        
495     claims live sheep cattle exports threatening tas
496             claims more rain will boost stock prices
497                       class action may cost esso 50m
498                        clinton cancels trip to india
499              coalition promises to scrap anglers fee
Name: headline_text, Length: 500, dtype: object

# **Named Entity Recognition**

Initialize training of model

In [4]:
# To get the config.cfg you need to follow the link https://spacy.io/usage/training
# and download the quickstart
!python -m spacy init fill-config base_config.cfg config.cfg

2022-12-05 04:29:27.551933: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Split the headlines into training and testing data with their classifiers 

In [5]:
# Load model from Spacy for named entity recognition and classification 
nlp=spacy.load("en_core_web_lg")

# Training data with classifiers
training=[]
# Testing data with classifiers
testing=[]

# Split 80% of headlines as training and 20% of headlines as testing
train,test=train_test_split(headlines,test_size=0.2)

for data in tqdm(nlp.pipe(train)):
  training.append(data)
for data in tqdm(nlp.pipe(test)):
  testing.append(data)

# Now we will store the required training data into the proper format for the model 
TRAIN_DATA=[]
for i in range(len(training)):
  for ent in training[i].ents:
    TRAIN_DATA.append((ent.text,[(ent.start_char,ent.end_char,ent.label_)]))
print(TRAIN_DATA)
# Same for testing data
TEST_DATA=[]
for i in range(len(testing)):
  for ent in training[i].ents:
    TEST_DATA.append((ent.text,[(ent.start_char,ent.end_char,ent.label_)]))
print(TEST_DATA)

400it [00:01, 281.16it/s]
100it [00:00, 405.28it/s]

[('hanson', [(0, 6, 'ORG')]), ('nsw', [(42, 45, 'ORG')]), ('new zealand', [(0, 11, 'GPE')]), ('kenya', [(24, 29, 'GPE')]), ('thursday', [(38, 46, 'DATE')]), ('thousands', [(0, 9, 'CARDINAL')]), ('61st', [(19, 23, 'ORDINAL')]), ('darwin', [(39, 45, 'PERSON')]), ('seven years', [(30, 41, 'DATE')]), ('nato', [(0, 4, 'ORG')]), ('turkey', [(33, 39, 'GPE')]), ('south korean', [(33, 45, 'NORP')]), ('patterson', [(0, 9, 'PERSON')]), ('crean', [(43, 48, 'PERSON')]), ('80', [(19, 21, 'CARDINAL')]), ('whyalla steelworks', [(26, 44, 'ORG')]), ('commonwealth bank', [(0, 17, 'ORG')]), ('esso', [(0, 4, 'ORG')]), ('korean', [(43, 49, 'GPE')]), ('one', [(29, 32, 'CARDINAL')]), ('wales', [(0, 5, 'ORG')]), ('italy', [(0, 5, 'GPE')]), ('ireland', [(27, 34, 'GPE')]), ('qld nats', [(0, 8, 'PERSON')]), ('libs springborg', [(28, 43, 'ORG')]), ('europe', [(33, 39, 'LOC')]), ('ge', [(45, 47, 'ORG')]), ('buenos', [(26, 32, 'GPE')]), ('murray meeting', [(0, 14, 'PERSON')]), ('indy', [(0, 4, 'PERSON')]), ('america

In [6]:
# Creating the spacy binary for training data which will be used for training 
db = DocBin() 
count=0
# Our own model with the train data
our_nlp=spacy.blank("en")
for text, annot in tqdm(TRAIN_DATA):
  doc = our_nlp.make_doc(text)  #make doc using text
  ents = []
  for start, end, label in annot: #create span object for entities
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      # print("Skipping entity")
      count+=1
    else:
        ents.append(span) #append span to entities
  doc.ents = ents 
  db.add(doc)

db.to_disk("train.spacy") 

print("\n"+str(count)+" entities have been skipped")

# Creating the spacy binary for testing data which will also be used along with the training data 
db = DocBin() 
count=0
# Our own model with the test data 
our_nlp=spacy.blank("en")
for text, annot in tqdm(TEST_DATA):
  doc = our_nlp.make_doc(text)  #make doc using text
  ents = []
  for start, end, label in annot: #create span object for entities
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      # print("Skipping entity")
      count+=1
    else:
        ents.append(span) #append span to entities
  doc.ents = ents 
  db.add(doc)

db.to_disk("test.spacy") 

print("\n"+str(count)+" entities have been skipped")

100%|██████████| 305/305 [00:00<00:00, 2580.16it/s]



172 entities have been skipped


100%|██████████| 87/87 [00:00<00:00, 5629.42it/s]


51 entities have been skipped


In [7]:
# Training pipeline
# Output of training is put into an output directory where we will use the best-model 
# If it takes too long just stop at when # is at least 1000 which means 1000 iterations were taken when training the data 
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy

2022-12-05 04:29:39.797785: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-12-05 04:29:40,639] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-05 04:29:40,654] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-12-05 04:29:40,658] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-05 04:29:40,660] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-12-05 04:29:40,787] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ne

In [8]:
# Display of using our model to categorize news headlines from the testing data 
our_nlp=spacy.load("output/model-best")
entities=[]

for data in tqdm(our_nlp.pipe(test,disable=["tagger", "parser"])):
  for ent in data.ents:
    entities.append((ent.text,ent.label_))
  displacy.render(data,style="ent",jupyter=True)
print(entities)
entity_dataframe=pd.DataFrame(entities,columns=["text","classifier"])

0it [00:00, ?it/s]/usr/local/lib/python3.8/dist-packages/spacy/displacy/__init__.py:206: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


13it [00:00, 127.84it/s]

48it [00:00, 254.48it/s]

74it [00:00, 248.69it/s]

100it [00:00, 259.62it/s]

[('sa govt', 'ORG'), ('saudi arabia', 'GPE'), ('aussie', 'NORP'), ('bowling change buchanan', 'ORG'), ('central qld hijack', 'ORG'), ('bartlett', 'NORP'), ('vows', 'NORP'), ('to block govt plans', 'ORG'), ('sa nuclear', 'PERSON'), ('sa', 'ORG'), ('abattoir up for sale', 'ORG'), ('again', 'GPE'), ('australia', 'GPE'), ('cairns', 'PERSON'), ('wins', 'ORG'), ('england', 'GPE'), ('three', 'CARDINAL'), ('wales match', 'ORG'), ('brigadier dismisses reports', 'ORG'), ('palaszczuk', 'NORP'), ('hands', 'ORG'), ('epa', 'ORG'), ('aboriginal tent embassy raid', 'ORG'), ('rain boosts farmers confidence', 'ORG'), ('qantas records', 'ORG'), ('anger grows after korean subway', 'ORG'), ('andersson', 'ORG'), ('atwal miss record', 'ORG'), ('a g calls', 'ORG'), ('hanson is', 'ORG'), ('iran', 'GPE'), ('crashes with', 'PERSON'), ('at least 250', 'CARDINAL'), ('families confront korean president', 'ORG'), ('at least 42', 'CARDINAL'), ('at richmond after', 'CARDINAL'), ('former bank worker pleads', 'ORG'), ('

In [9]:
# Dataframe which holds the text and its classifier
entity_dataframe

,text,classifier
0,sa govt,ORG
1,saudi arabia,GPE
2,aussie,NORP
3,bowling change buchanan,ORG
4,central qld hijack,ORG
5,bartlett,NORP
6,vows,NORP
7,to block govt plans,ORG
8,sa nuclear,PERSON
9,sa,ORG


In [10]:
# Shows occurences of classifiers from news headlines 
class_occur=entity_dataframe.groupby(["classifier"]).count()
class_occur

,text
classifier,
CARDINAL,4
GPE,5
NORP,4
ORG,23
PERSON,3


In [12]:
# We evaluate our model based on the following scores: 

# NER P or the precision of our model is around 94%
# NER R or the recall is 96.88 
# NER F or the F-score of our model is around 95% 

!python -m spacy evaluate "output/model-best" "test.spacy" --gold-preproc 

2022-12-05 04:31:16.132885: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   87.50 
NER R   97.22 
NER F   92.11 
SPEED   7122  


=============================== NER (per type) ===============================

                P        R       F
ORG         84.62   100.00   91.67
GPE         80.00   100.00   88.89
CARDINAL    75.00   100.00   85.71
PERSON     100.00    93.33   96.55
NORP        75.00   100.00   85.71

